In [ ]:
# %% Deep learning - Section 18.163
#    Convolution in code

# This code pertains a deep learning course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/deeplearning_x
# The "base" code in this repository is adapted (with very minor modifications)
# from code developed by the course instructor (Mike X. Cohen), while the
# "exercises" and the "code challenges" contain more original solutions and
# creative input from my side. If you are interested in DL (and if you are
# reading this statement, chances are that you are), go check out the course, it
# is singularly good.


In [61]:
# %% Libraries and modules
import numpy               as np
import matplotlib.pyplot   as plt
import torch
import torch.nn            as nn
import seaborn             as sns
import copy
import torch.nn.functional as F
import pandas              as pd
import scipy.stats         as stats
import sklearn.metrics     as skm
import time
import sys
import imageio.v2          as imageio

from torch.utils.data                 import DataLoader,TensorDataset
from sklearn.model_selection          import train_test_split
from google.colab                     import files
from torchsummary                     import summary
from scipy.stats                      import zscore
from sklearn.decomposition            import PCA
from scipy.signal                     import convolve2d
from IPython                          import display
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')
plt.style.use('default')

In [ ]:
# %% Manual 2D convolution in numpy

# Image and kernel (Gaussian)
img_n = 20
image = np.random.randn(img_n,img_n)

kernel_n = 7
X,Y      = np.meshgrid(np.linspace(-3,3,kernel_n),np.linspace(-3,3,kernel_n))
kernel   = np.exp(-(X**2 + Y**2)/7)

# Plotting
phi = (1 + np.sqrt(5))/2
fig,ax = plt.subplots(1,2,figsize=(phi*5,5))

ax[0].imshow(image)
ax[0].set_title('Image')

ax[1].imshow(kernel)
ax[1].set_title('Kernel')

plt.savefig('figure1_convolution.png')
plt.show()
files.download('figure1_convolution.png')


In [11]:
# %% Implement manual convolution

convolve_output_1 = np.zeros((img_n,img_n))
half_kernel       = kernel_n//2

for row in range(half_kernel,img_n-half_kernel):
    for col in range(half_kernel,img_n-half_kernel):

        # Get current piece of image (select rows of interest first, and then
        # intersect with cols of interest)
        image_chunk = image[ row-half_kernel:row+half_kernel+1, : ]
        image_chunk = image_chunk[ :, col-half_kernel:col+half_kernel+1 ]

        # Dot product between kernel and image chunk (flip the kernel for real
        # convolution, otherwise it's a cross-correlation; not super important
        # here with a gaussian - symmetric - filter, but just fyi)
        dot_prod = np.sum( image_chunk*kernel[::-1,::-1] )

        # Store
        convolve_output_1[row,col] = dot_prod


In [7]:
# %% 2D convolution in scipy

convolve_output_2 = convolve2d(image,kernel,mode='valid')


In [ ]:
# %% Plotting

phi = (1 + np.sqrt(5))/2
fig,ax = plt.subplots(2,2,figsize=(phi*6,6))

ax[0,0].imshow(image)
ax[0,0].set_title('Image')

ax[0,1].imshow(kernel)
ax[0,1].set_title('Convolution kernel')

ax[1,0].imshow(convolve_output_1)
ax[1,0].set_title('Manual convolution')

ax[1,1].imshow(convolve_output_2)
ax[1,1].set_title("Scipy's convolution")

plt.tight_layout()

plt.savefig('figure2_convolution.png')
plt.show()
files.download('figure2_convolution.png')



In [ ]:
# %% Example with real images

# Image
nimma = imageio.imread('https://www.patrickvanos.com/img/full/stevenskerk-nijmegen-the-netherlands.jpg')
nimma = imageio.imread('https://rijkswaterstaat.imgix.net/waalbrug_tcm26-337226.jpg?crop=entropy')
nimma = imageio.imread('https://images.squarespace-cdn.com/content/v1/55ce8aece4b02774e51c931f/1462813252035-UFQZXZ7A3YUP856STD05/beauvais+cathedral+height')
nimma = imageio.imread('https://800anscathedrale.beauvais.fr/wp-content/uploads/2025/01/img-1789-2560x1920.jpg')

# uploaded = files.upload()
# nimma = imageio.imread(list(uploaded.keys())[0])

print(nimma.shape)

phi = (1 + np.sqrt(5))/2
fig = plt.figure(figsize=(phi*5,5))
plt.imshow(nimma)

plt.savefig('figure3_convolution.png')
plt.show()
files.download('figure3_convolution.png')

# Compress image in 2D for convenience
nimma = np.mean(nimma,axis=2)
nimma = nimma/np.max(nimma)

phi = (1 + np.sqrt(5))/2
fig = plt.figure(figsize=(phi*5,5))
plt.imshow(nimma,cmap='gray')

plt.savefig('figure4_convolution.png')
plt.show()
files.download('figure4_convolution.png')


In [70]:
# %% Hand-crafted convolution kernels

# Vertical kernel (note how the kernels sum to zero, so to avoid the kernel
# adding an offset to the resulting feature map)
vk = np.array([ [-1,0,1],
                [-1,0,1],
                [-1,0,1] ])

# Horizontal kernel
hk = np.array([ [-1,-1,-1],
                [ 0, 0, 0],
                [ 1, 1, 1] ])


In [ ]:
# %% Plotting

phi = (1 + np.sqrt(5)) / 2
fig,ax = plt.subplots(2,2,figsize=(phi*6,6))

ax[0,0].imshow(vk,cmap='gray')
ax[0,0].set_title('Vertical kernel')

ax[0,1].imshow(hk,'gray')
ax[0,1].set_title('Horizontal kernel')

conv = convolve2d(nimma,vk,mode='same')
ax[1,0].imshow(conv,cmap='gray',vmin=0,vmax=.01)

conv = convolve2d(nimma,hk,mode='same')
ax[1,1].imshow(conv,cmap='gray',vmin=0,vmax=.01)

plt.tight_layout()

plt.savefig('figure5_convolution.png')
plt.show()
files.download('figure5_convolution.png')


In [ ]:
# %% # %% 2D convolution in PyTorch

# view(a,b,c,d) are the kernel's params;
# a = num of img in batch; b = channels; c = height; d = width
vk_torch = torch.tensor(vk).view(1,1,3,3).double()
hk_torch = torch.tensor(hk).view(1,1,3,3).double()
nimma_t  = torch.tensor(nimma).view(1,1,nimma.shape[0],nimma.shape[1])

print(vk_torch.shape)
print(hk_torch.shape)
print(nimma_t.shape)

conv_torch = F.conv2d(nimma_t,vk_torch)
print(conv_torch.shape)


In [ ]:
# %% Plotting

phi = (1 + np.sqrt(5)) / 2
fig,ax = plt.subplots(2,2,figsize=(phi*6,6))

img = torch.squeeze(vk_torch.detach())
ax[0,0].imshow(img,cmap='gray')
ax[0,0].set_title('Vertical kernel')

img = torch.squeeze(hk_torch.detach())
ax[0,1].imshow(img,'gray')
ax[0,1].set_title('Horizontal kernel')

conv = F.conv2d(nimma_t,vk_torch)
img  = torch.squeeze(conv.detach())
ax[1,0].imshow(img,cmap='gray',vmin=0,vmax=.01)

conv = F.conv2d(nimma_t,hk_torch)
img  = torch.squeeze(conv.detach())
ax[1,1].imshow(img,cmap='gray',vmin=0,vmax=.01)

plt.tight_layout()

plt.savefig('figure18_convolution.png')
plt.show()
files.download('figure18_convolution.png')


In [98]:
# %% Exercise 1
#    Try creating your own convolution kernels! It's fun and easy ;)
#    Note that image kernels are often crafted to sum to zero, which prevents
#    a global shift in the result. But you don't need to follow that convention.

# Keep var names for ease
vk = np.array([ [-2, 0, 0, 0, 2],
                [-2, 0, 0, 0, 2],
                [-2, 0, 0, 0, 2],
                [-2, 0, 0, 0, 2],
                [-2, 0, 0, 0, 2] ])

hk = np.array([ [-2,-2,-2,-2,-2],
                [ 0, 0, 0, 0, 0],
                [ 0, 0, 0, 0, 0],
                [ 0, 0, 0, 0, 0],
                [ 2, 2, 2, 2, 2] ])


In [110]:
# %% Exercise 1
#    Continue ...

# Keep var names for ease
vk = np.array([ [-1, 0, 0, 0, 1],
                [-2, 0, 0, 0, 2],
                [-4, 0, 0, 0, 4],
                [-2, 0, 0, 0, 2],
                [-1, 0, 0, 0, 1] ])

hk = np.array([ [  0,-1, 0],
                [ -1, 5,-1],
                [  0,-1, 0] ])
